In [10]:
import torch

# Create a random tensor of size [10, 512]
random_tensor = torch.rand(10, 512)

# Create a tensor of twos of size [10, 10]
tensor_of_twos = 2 * torch.ones(10, 10)

# Perform matrix multiplication
result = torch.matmul(tensor_of_twos, random_tensor)

print(f"Random Tensor: {random_tensor.shape}")
# print(random_tensor)

print(f"\nTensor of Twos: {tensor_of_twos.shape}")
# print(tensor_of_twos)

print(f"\nResult of Matmul: {result.shape}")
# print(result)


Random Tensor: torch.Size([10, 512])

Tensor of Twos: torch.Size([10, 10])

Result of Matmul: torch.Size([10, 512])
